## Initializing colab
The two cells below are used only in case this notebook is executed via **Google Colab**. Take into account that, for running conda on **Google Colab**, the **condacolab** library must be installed. As [explained here](https://pypi.org/project/condacolab/), the installation requires a **kernel restart**, so when running this notebook in **Google Colab**, don't run all cells until this **installation** is properly **finished** and the **kernel** has **restarted**.

In [1]:
# Only executed when using google colab
import sys
if 'google.colab' in sys.modules:
  import subprocess
  from pathlib import Path
  try:
    subprocess.run(["conda", "-V"], check=True)
  except FileNotFoundError:
    subprocess.run([sys.executable, "-m", "pip", "install", "condacolab"], check=True)
    import condacolab
    condacolab.install()
    # Clone repository
    repo_URL = "https://github.com/RubenChM/biobb_wf_haddock.git"
    repo_name = Path(repo_URL).name.split('.')[0]
    if not Path(repo_name).exists():
      subprocess.run(["mamba", "install", "-y", "git"], check=True)
      subprocess.run(["git", "clone", repo_URL], check=True)
      print("⏬ Repository properly cloned.")
    # Install environment
    print("⏳ Creating environment...")
    env_file_path = f"{repo_name}/conda_env/haddock.yml"
    subprocess.run(["mamba", "env", "update", "-n", "base", "-f", env_file_path], check=True)
    print("🎨 Install NGLView dependencies...")
    subprocess.run(["mamba", "install", "-y", "-c", "conda-forge", "nglview==3.1.4", "ipywidgets==8.1.6"], check=True)
    print("👍 Conda environment successfully created and updated.")

In [ ]:
# Enable widgets for colab
if 'google.colab' in sys.modules:
  from google.colab import output
  output.enable_custom_widget_manager()
  # Change working dir
  import os
  os.chdir("biobb_wf_haddock/biobb_wf_haddock/notebooks")
  print(f"📂 New working directory: {os.getcwd()}")

# Helpers

In [1]:
# TO BE REMOVED!!!
%load_ext autoreload
%autoreload 2

# Imports
import os, shutil
import nglview as nv
import ipywidgets
import pandas as pd
import zipfile
import webbrowser

# Helpers
def def_dict(propierties={}):
    def_props = {'out_log_path': 'log/log.log',
                 'err_log_path': 'log/log.err',
                 'remove_tmp': True,
                 'can_write_console_log': False}
    def_props.update(propierties)
    return def_props

def show_pdbs(pdbs, surface=False):
    # Load the PDB files
    views = [nv.show_file(pdb) for pdb in pdbs]
    for view in views:
        if surface:
            view.clear()
            view.add_cartoon(color='black')
            view.add_surface(color='electrostatic', opacity=0.5)
        view.layout.width = 'auto'
        view.layout.max_width = f'{100/len(views)}%'
    return ipywidgets.HBox(views)

def display_actpass(pdb, actpass, opacity=1):
    with open(actpass, 'r') as file:
        actpass = file.read().splitlines()
        act_res = actpass[0].replace(' ', ', ')
        pas_res = actpass[1].replace(' ', ', ')
        
    # Load the PDB files
    view = nv.NGLWidget()
    view.add_component(pdb)
    view.clear()
    view.add_cartoon(color='black')
    view.add_ball_and_stick(color='grey',opacity=opacity)
    view.add_surface(selection=f'not ( {pas_res}, {act_res} )', color='white', opacity=opacity)
    if act_res != '':
        view.add_surface(selection=f'{act_res}', color='red')
    if pas_res != '':
        view.add_surface(selection=f'{pas_res}', color='green', opacity=opacity)
    view.layout.width = '100%'
    return view

# Haddock3 protein protein docking using BioExcel Building Blocks (biobb)
***
This tutorial aims to illustrate the process of **proptein protein docking** using **Haddock3**, step by step, using the **BioExcel Building Blocks library (biobb)**. 
***
**Biobb modules** used:

 - [biobb_haddock](https://github.com/bioexcel/biobb_haddock): Biobb building blocks for the Haddock3 suite.
 
**Auxiliar libraries** used:

 - [nb_conda_kernels](https://github.com/Anaconda-Platform/nb_conda_kernels): Enables a Jupyter Notebook or JupyterLab application in one conda environment to access kernels for Python, R, and other languages found in other environments.
 - [nglview](http://nglviewer.org/#nglview): Jupyter/IPython widget to interactively view molecular structures and trajectories in notebooks.
 - [ipywidgets](https://github.com/jupyter-widgets/ipywidgets): Interactive HTML widgets for Jupyter notebooks and the IPython kernel.

### Conda Installation and Launch

```console
git clone https://github.com/bioexcel/biobb_wf_ligand_parameterization.git
cd biobb_wf_ligand_parameterization
conda env create -f conda_env/environment.yml
conda activate biobb_ligand_parameterization_tutorial
jupyter-nbextension enable --py --user widgetsnbextension
jupyter-nbextension enable --py --user nglview
jupyter-notebook biobb_wf_ligand_parameterization/notebooks/biobb_ligand_parameterization_tutorial.ipynb
  ``` 

***
### Pipeline steps:
 1. [Input Parameters](#input)
 2. [Create topology](#fetch)
 3. [CAPRI evaluation](#addh)
 4. [Select Top structures](#min)
 5. [Flexible Refinement](#acpype)
 6. [2nd CAPRI evalutation](#output)
 7. [Energy Minimization Refinement](#questions)
 8. [Energy Minimization Refinement](#questions)
 
***
![](https://bioexcel.eu/wp-content/uploads/2019/04/Bioexcell_logo_1080px_transp.png)
***

<a id="input"></a>
***
## Input parameters
**Input parameters** needed:
 - **ligandCode**: 3-letter code of the ligand structure (e.g. IBP)
 - **mol_charge**: Molecule net charge (e.g. -1)
 - **pH**: Acidity or alkalinity for the small molecule. Hydrogen atoms will be added according to this pH. (e.g. 7.4)

# HADDOCK3 PPI tutorial

In [37]:
# https://www.bonvinlab.org/education/HADDOCK3/HADDOCK3-antibody-antigen/#setting-up-and-running-the-docking-with-haddock3
# data from: https://surfdrive.surf.nl/files/index.php/s/R7VHGQM9nx8QuQn

ab_id    = '4G6K'  # antibody
ag_id    = '4I1B' # antigen
ref_id   = '4G6M' 
out_path = './data/antibody/'
data_pth = '/home/rchaves/repo/biobb_haddock/biobb_haddock/test/data/haddock/'

In [18]:
os.makedirs('data',exist_ok=True)
os.makedirs(out_path,exist_ok=True)
os.makedirs(f'{out_path}pre',exist_ok=True)
os.makedirs(f'{out_path}docking',exist_ok=True)

## Preparing PDB files for docking

### Fetching the PDBs

In [19]:
# Downloading desired PDB files
# Import module
from biobb_io.api.pdb import pdb

# Create properties dict and inputs/outputs
ab_pdb  = f'{out_path}/pre/{ab_id}_0.pdb'
ag_pdb  = f'{out_path}/pre/{ag_id}_0.pdb'
ref_pdb = f'{out_path}/pre/{ref_id}_0.pdb'

# Create and launch bb
pdb(output_pdb_path = ab_pdb,  properties = def_dict({'pdb_code': ab_id}))
pdb(output_pdb_path = ag_pdb,  properties = def_dict({'pdb_code': ag_id}))
pdb(output_pdb_path = ref_pdb, properties = def_dict({'pdb_code': ref_id}))

2025-05-12 17:05:52,735 [MainThread  ] [INFO ]  Module: biobb_io.api.pdb Version: 5.0.1
2025-05-12 17:05:52,736 [MainThread  ] [INFO ]  Downloading 4g6k from: https://www.ebi.ac.uk/pdbe/entry-files/download/pdb4g6k.ent
2025-05-12 17:05:53,072 [MainThread  ] [INFO ]  Writting pdb to: ./data/antibody//pre/4G6K_0.pdb
2025-05-12 17:05:53,073 [MainThread  ] [INFO ]  Filtering lines NOT starting with one of these words: ['ATOM', 'MODEL', 'ENDMDL']
2025-05-12 17:05:53,076 [MainThread  ] [INFO ]  
2025-05-12 17:05:53,078 [MainThread  ] [INFO ]  Module: biobb_io.api.pdb Version: 5.0.1
2025-05-12 17:05:53,079 [MainThread  ] [INFO ]  Downloading 4i1b from: https://www.ebi.ac.uk/pdbe/entry-files/download/pdb4i1b.ent
2025-05-12 17:05:53,287 [MainThread  ] [INFO ]  Writting pdb to: ./data/antibody//pre/4I1B_0.pdb
2025-05-12 17:05:53,287 [MainThread  ] [INFO ]  Filtering lines NOT starting with one of these words: ['ATOM', 'MODEL', 'ENDMDL']
2025-05-12 17:05:53,288 [MainThread  ] [INFO ]  
2025-05-12

0

### Preparing the antibody structure

In [20]:
from biobb_pdb_tools.pdb_tools.biobb_pdb_tidy import biobb_pdb_tidy
from biobb_pdb_tools.pdb_tools.biobb_pdb_selchain import biobb_pdb_selchain
from biobb_pdb_tools.pdb_tools.biobb_pdb_delhetatm import biobb_pdb_delhetatm
from biobb_pdb_tools.pdb_tools.biobb_pdb_fixinsert import biobb_pdb_fixinsert
from biobb_pdb_tools.pdb_tools.biobb_pdb_selaltloc import biobb_pdb_selaltloc
from biobb_pdb_tools.pdb_tools.biobb_pdb_keepcoord import biobb_pdb_keepcoord
from biobb_pdb_tools.pdb_tools.biobb_pdb_selres import biobb_pdb_selres

steps = [
    biobb_pdb_tidy,
    biobb_pdb_selchain,
    biobb_pdb_delhetatm,
    biobb_pdb_fixinsert,
    biobb_pdb_selaltloc,
    biobb_pdb_keepcoord,
    biobb_pdb_selres,
    biobb_pdb_tidy,
]

for ch in ['H','L']:
    step_props = {
        'biobb_pdb_tidy':     {'strict': True},
        'biobb_pdb_selchain': {'chains': ch},
        'biobb_pdb_selres':   {'selection': f'1:{ 120 if ch == "H" else 107}'},
    }
    for i, step in enumerate(steps):
        pdb_in  = f'{out_path}/pre/{ab_id}_{i}.pdb'
        if i+1 < len(steps):
            pdb_out  = f'{out_path}/pre/{ab_id}_{i+1}.pdb'
        else:
            pdb_out  = f'{out_path}/pre/{ab_id}_{ch}.pdb'
        props = def_dict(step_props.get(step.__name__, {}))
        step(input_file_path = pdb_in,  output_file_path=pdb_out,  properties = props)

2025-05-12 17:05:53,618 [MainThread  ] [INFO ]  Module: biobb_pdb_tools.pdb_tools.biobb_pdb_tidy Version: 5.0.1
2025-05-12 17:05:53,618 [MainThread  ] [INFO ]  /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_a29838d2-0584-4636-9c64-7fc786c45cef directory successfully created
2025-05-12 17:05:53,619 [MainThread  ] [INFO ]  Copy: ./data/antibody//pre/4G6K_0.pdb to /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_a29838d2-0584-4636-9c64-7fc786c45cef
2025-05-12 17:05:53,619 [MainThread  ] [INFO ]  Appending optional boolean property
2025-05-12 17:05:53,619 [MainThread  ] [INFO ]  pdb_tidy -strict ./data/antibody//pre/4G6K_0.pdb > ./data/antibody//pre/4G6K_1.pdb
2025-05-12 17:05:53,620 [MainThread  ] [INFO ]  Creating command line with instructions and required arguments
2025-05-12 17:05:53,620 [MainThread  ] [INFO ]  pdb_tidy -strict ./data/antibody//pre/4G6K_0.pdb > ./data/antibody//pre/4G6K_1.pdb

2025-05-12 17:05:53,643 [M

In [21]:
import zipfile

# Define the zip file path
zip_file_path = f'{out_path}/pre/{ab_id}_HL.zip'

# Create a zip file and add the pdb_out file to it
with zipfile.ZipFile(zip_file_path, 'w') as zipf:
    zipf.write(f'{out_path}/pre/{ab_id}_H.pdb', arcname=f'{ab_id}_H.pdb')
    zipf.write(f'{out_path}/pre/{ab_id}_L.pdb', arcname=f'{ab_id}_L.pdb')

In [22]:
from biobb_pdb_tools.pdb_tools.biobb_pdb_merge import biobb_pdb_merge
from biobb_pdb_tools.pdb_tools.biobb_pdb_reres import biobb_pdb_reres
from biobb_pdb_tools.pdb_tools.biobb_pdb_chain import biobb_pdb_chain
from biobb_pdb_tools.pdb_tools.biobb_pdb_chainxseg import biobb_pdb_chainxseg

steps = [
    biobb_pdb_merge,
    biobb_pdb_reres,
    biobb_pdb_chain,
    biobb_pdb_chainxseg,
    biobb_pdb_tidy,
]

step_props = {
    'pdb_reres': {'number': 1},
    'biobb_pdb_chain': {'chain': 'A'},
    'biobb_pdb_tidy': {'strict': True},
}

for i, step in enumerate(steps):
    pdb_in  = (zip_file_path if i == 0 
               else f'{out_path}/pre/{ab_id}_HL_{i}.pdb')
    
    pdb_out = (f'{out_path}/pre/{ab_id}_HL_{i+1}.pdb' 
               if i+1 < len(steps) 
               else f'{out_path}/{ab_id}_clean.pdb')
    
    props = def_dict(step_props.get(step.__name__, {}))
    step(input_file_path = pdb_in,  output_file_path=pdb_out,  properties = props)

2025-05-12 17:05:54,063 [MainThread  ] [INFO ]  Module: biobb_pdb_tools.pdb_tools.biobb_pdb_merge Version: 5.0.1
2025-05-12 17:05:54,064 [MainThread  ] [INFO ]  /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_6b9f1c0d-121c-49a7-a299-21f5497dd888 directory successfully created
2025-05-12 17:05:54,065 [MainThread  ] [INFO ]  Copy: ./data/antibody//pre/4G6K_HL.zip to /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_6b9f1c0d-121c-49a7-a299-21f5497dd888
2025-05-12 17:05:54,066 [MainThread  ] [INFO ]  pdb_merge /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_6b9f1c0d-121c-49a7-a299-21f5497dd888/4G6K_H.pdb /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_6b9f1c0d-121c-49a7-a299-21f5497dd888/4G6K_L.pdb > ./data/antibody//pre/4G6K_HL_1.pdb
2025-05-12 17:05:54,066 [MainThread  ] [INFO ]  Creating command line with instructions and required arguments
2025-05-12 17:05:5

### Preparing the antigen structure

In [23]:
steps = [
    biobb_pdb_tidy,
    biobb_pdb_delhetatm,
    biobb_pdb_selaltloc,
    biobb_pdb_keepcoord,
    biobb_pdb_chain,
    biobb_pdb_chainxseg,
    biobb_pdb_tidy,
]

step_props = {
    'biobb_pdb_tidy': {'strict': True},
    'biobb_pdb_chain': {'chain': 'B'},
}

for i, step in enumerate(steps):
    pdb_in  = f'{out_path}/pre/{ag_id}_{i}.pdb'
    pdb_out = (f'{out_path}/pre/{ag_id}_{i+1}.pdb' 
               if i+1 < len(steps) 
               else f'{out_path}/{ag_id}_clean.pdb')
    props = def_dict(step_props.get(step.__name__, {}))
    step(input_file_path = pdb_in,  output_file_path=pdb_out,  properties = props)

2025-05-12 17:05:54,206 [MainThread  ] [INFO ]  Module: biobb_pdb_tools.pdb_tools.biobb_pdb_tidy Version: 5.0.1
2025-05-12 17:05:54,207 [MainThread  ] [INFO ]  /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_2bf7c004-59ee-4a16-bb36-9a7a0bbe3dab directory successfully created
2025-05-12 17:05:54,208 [MainThread  ] [INFO ]  Copy: ./data/antibody//pre/4I1B_0.pdb to /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_2bf7c004-59ee-4a16-bb36-9a7a0bbe3dab
2025-05-12 17:05:54,208 [MainThread  ] [INFO ]  Appending optional boolean property
2025-05-12 17:05:54,208 [MainThread  ] [INFO ]  pdb_tidy -strict ./data/antibody//pre/4I1B_0.pdb > ./data/antibody//pre/4I1B_1.pdb
2025-05-12 17:05:54,209 [MainThread  ] [INFO ]  Creating command line with instructions and required arguments
2025-05-12 17:05:54,209 [MainThread  ] [INFO ]  pdb_tidy -strict ./data/antibody//pre/4I1B_0.pdb > ./data/antibody//pre/4I1B_1.pdb

2025-05-12 17:05:54,228 [M

### Preparing the reference pdb

In [24]:
steps = [
    biobb_pdb_tidy,
    biobb_pdb_selchain
]

step_props = {
    'biobb_pdb_tidy': {'strict': True},
    'biobb_pdb_selchain': {'chains': 'H,L'},
}

for i, step in enumerate(steps):
    pdb_in  = f'{out_path}pre/{ref_id}_{i}.pdb'
    pdb_out = f'{out_path}pre/{ref_id}_{i+1}.pdb' 
    props = def_dict(step_props.get(step.__name__, {}))
    step(input_file_path = pdb_in,  output_file_path=pdb_out,  properties = props)

2025-05-12 17:05:54,394 [MainThread  ] [INFO ]  Module: biobb_pdb_tools.pdb_tools.biobb_pdb_tidy Version: 5.0.1
2025-05-12 17:05:54,394 [MainThread  ] [INFO ]  /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_c6b097d1-b0d7-477f-a8be-a587383ec429 directory successfully created
2025-05-12 17:05:54,396 [MainThread  ] [INFO ]  Copy: ./data/antibody/pre/4G6M_0.pdb to /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_c6b097d1-b0d7-477f-a8be-a587383ec429
2025-05-12 17:05:54,396 [MainThread  ] [INFO ]  Appending optional boolean property
2025-05-12 17:05:54,396 [MainThread  ] [INFO ]  pdb_tidy -strict ./data/antibody/pre/4G6M_0.pdb > ./data/antibody/pre/4G6M_1.pdb
2025-05-12 17:05:54,397 [MainThread  ] [INFO ]  Creating command line with instructions and required arguments
2025-05-12 17:05:54,397 [MainThread  ] [INFO ]  pdb_tidy -strict ./data/antibody/pre/4G6M_0.pdb > ./data/antibody/pre/4G6M_1.pdb

2025-05-12 17:05:54,418 [MainTh

In [25]:
steps = [
    biobb_pdb_tidy,
    biobb_pdb_selchain,
    biobb_pdb_delhetatm,
    biobb_pdb_fixinsert,
    biobb_pdb_selaltloc,
    biobb_pdb_keepcoord,
    biobb_pdb_selres,
    biobb_pdb_tidy,
]
sels = {"H": 120, "L":107, "A": ''}

for ch in ['H','L']:
    step_props = {
        'biobb_pdb_tidy':     {'strict': True},
        'biobb_pdb_selchain': {'chains': ch},
        'biobb_pdb_selres':   {'selection': f'1:{sels[ch]}'}
    }
    for i, step in enumerate(steps):
        pdb_in  = f'{out_path}/pre/{ref_id}_{i}.pdb'
        if i+1 < len(steps):
            pdb_out  = f'{out_path}/pre/{ref_id}_{i+1}.pdb'
        else:
            pdb_out  = f'{out_path}/pre/{ref_id}_{ch}.pdb'
        props = def_dict(step_props.get(step.__name__, {}))
        step(input_file_path = pdb_in,  output_file_path=pdb_out,  properties = props)

2025-05-12 17:05:54,498 [MainThread  ] [INFO ]  Module: biobb_pdb_tools.pdb_tools.biobb_pdb_tidy Version: 5.0.1
2025-05-12 17:05:54,499 [MainThread  ] [INFO ]  /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_c5f4f3e3-fa33-46bb-85fc-5c73f5f3e012 directory successfully created
2025-05-12 17:05:54,500 [MainThread  ] [INFO ]  Copy: ./data/antibody//pre/4G6M_0.pdb to /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_c5f4f3e3-fa33-46bb-85fc-5c73f5f3e012
2025-05-12 17:05:54,500 [MainThread  ] [INFO ]  Appending optional boolean property
2025-05-12 17:05:54,501 [MainThread  ] [INFO ]  pdb_tidy -strict ./data/antibody//pre/4G6M_0.pdb > ./data/antibody//pre/4G6M_1.pdb
2025-05-12 17:05:54,501 [MainThread  ] [INFO ]  Creating command line with instructions and required arguments
2025-05-12 17:05:54,501 [MainThread  ] [INFO ]  pdb_tidy -strict ./data/antibody//pre/4G6M_0.pdb > ./data/antibody//pre/4G6M_1.pdb

2025-05-12 17:05:54,524 [M

In [26]:
ref_H = f'{out_path}/pre/{ref_id}_H.pdb'
ref_L = f'{out_path}/pre/{ref_id}_L.pdb'
ref_HL = f'{out_path}/pre/{ref_id}_HL.pdb'

!pdb_merge {ref_H} {ref_L} | pdb_reres -1 | pdb_chain -A | pdb_chainxseg | pdb_tidy -strict > {ref_HL}

In [27]:
pdb_in  = f'{out_path}/pre/{ref_id}_0.pdb'
ref_A = f'{out_path}/pre/{ref_id}_A.pdb'
ref_pdb_clean = f'{out_path}/{ref_id}_clean.pdb'

!pdb_selchain -A {pdb_in} | pdb_reatom -0 | pdb_chain -B | pdb_chainxseg > {ref_A}
!pdb_merge {ref_HL} {ref_A} | pdb_segxchain | pdb_tidy -strict > {ref_pdb_clean}

## Defining restrains

#### Paratope
The residues of the hypervariable loops involved in binding. Identified with:
- [ProABC-2](https://academic.oup.com/bioinformatics/article/36/20/5107/5873593?login=false)

#### Epitope

From [bibliography](https://linkinghub.elsevier.com/retrieve/pii/S0022283612007863)

In [28]:
paratope_sel = '31,32,33,34,35,52,54,55,56,100,101,102,103,104,105,106,151,152,169,170,173,211,212,213,214,216'
epitope_sel  = '72,73,74,75,81,83,84,89,90,92,94,96,97,98,115,116,117'

In [55]:
ab_pdb_clean = f'{out_path}/{ab_id}_clean.pdb'
ag_pdb_clean = f'{out_path}/{ag_id}_clean.pdb'
ref_pdb_clean = f'{out_path}/{ref_id}_clean.pdb'
views = show_pdbs([ab_pdb_clean, ag_pdb_clean, ref_pdb_clean])

In [53]:
views.children[0].add_surface(selection=paratope_sel.replace(',', ', '), color='red')
views.children[1].add_surface(selection=epitope_sel.replace(',', ', '), color='red')
views

In [31]:
# Obtain passive from active selection
from biobb_haddock.haddock_restraints.haddock3_passive_from_active import haddock3_passive_from_active

ab_actpass = f'{out_path}{ab_id}_actpass.txt'
ag_actpass = f'{out_path}{ag_id}_actpass.txt'

# Create the actpass for the antibody manually
with open(ab_actpass, 'w') as f:
    f.write( paratope_sel.replace(',', ' ')+'\n\n')

# For the antigen, we will use the epitope selection as the active selection
# and some reidues around it as passsive
haddock3_passive_from_active( 
    input_pdb_path      = ag_pdb_clean,
    output_actpass_path = ag_actpass,
    properties          = def_dict({'active_list' : epitope_sel}))

2025-05-12 17:05:55,983 [MainThread  ] [INFO ]  Module: biobb_haddock.haddock_restraints.haddock3_passive_from_active Version: 5.0.1
2025-05-12 17:05:55,984 [MainThread  ] [INFO ]  /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_ace851cd-fdf1-46cc-a6d4-fd6cf9bada73 directory successfully created
2025-05-12 17:05:55,985 [MainThread  ] [INFO ]  Copy: ./data/antibody//4I1B_clean.pdb to /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_ace851cd-fdf1-46cc-a6d4-fd6cf9bada73
2025-05-12 17:05:55,985 [MainThread  ] [INFO ]  haddock3-restraints passive_from_active /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_ace851cd-fdf1-46cc-a6d4-fd6cf9bada73/4I1B_clean.pdb 72,73,74,75,81,83,84,89,90,92,94,96,97,98,115,116,117 &> /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_ace851cd-fdf1-46cc-a6d4-fd6cf9bada73/4I1B_actpass.txt

2025-05-12 17:05:56,486 [MainThread  ] [INFO ]  E

0

In [32]:
display_actpass(ag_pdb_clean, ag_actpass)

NGLWidget(layout=Layout(width='100%'))

#### Defining ambiguous restraints

In [33]:
# Convert active/passive to ambiguous restraints
from biobb_haddock.haddock_restraints.haddock3_actpass_to_ambig import haddock3_actpass_to_ambig

complex_tbl = f'{out_path}ambig-paratope-NMR-epitope.tbl'

haddock3_actpass_to_ambig( 
    input_actpass1_path=ab_actpass,
    input_actpass2_path=ag_actpass,    
    output_tbl_path=complex_tbl,
    properties = def_dict({
        'segid_one': 'A', 
        'segid_two': 'B'}))

2025-05-12 17:05:56,642 [MainThread  ] [INFO ]  Module: biobb_haddock.haddock_restraints.haddock3_actpass_to_ambig Version: 5.0.1
2025-05-12 17:05:56,642 [MainThread  ] [INFO ]  /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_5cef71a6-172d-4620-9fd2-ddef3ee770d3 directory successfully created
2025-05-12 17:05:56,643 [MainThread  ] [INFO ]  Copy: ./data/antibody/4G6K_actpass.txt to /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_5cef71a6-172d-4620-9fd2-ddef3ee770d3
2025-05-12 17:05:56,643 [MainThread  ] [INFO ]  Copy: ./data/antibody/4I1B_actpass.txt to /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_5cef71a6-172d-4620-9fd2-ddef3ee770d3
2025-05-12 17:05:56,644 [MainThread  ] [INFO ]  haddock3-restraints active_passive_to_ambig /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_5cef71a6-172d-4620-9fd2-ddef3ee770d3/4G6K_actpass.txt /home/rchaves/repo/ab_design/b

2025-05-12 17:05:57,001 [MainThread  ] [INFO ]  Executing: haddock3-restraints active_passive_to_ambig /home/rchaves/repo/ab_design/biobb_w...
2025-05-12 17:05:57,002 [MainThread  ] [INFO ]  Exit code: 0
2025-05-12 17:05:57,003 [MainThread  ] [INFO ]  Removed: ['/home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_5cef71a6-172d-4620-9fd2-ddef3ee770d3']


0

In [34]:
# Validate tbl
!haddock3-restraints validate_tbl {complex_tbl} --silent

#### Additional restraints for multi-chain proteins

In [35]:
# Tie antibody chains together
from biobb_haddock.haddock_restraints.haddock3_restrain_bodies import haddock3_restrain_bodies

body_tbl = f'{out_path}antibody-unambig.tbl'

haddock3_restrain_bodies( 
    input_structure_path=ab_pdb_clean,
    output_tbl_path=body_tbl,
    properties = def_dict())

2025-05-12 17:05:57,619 [MainThread  ] [INFO ]  Module: biobb_haddock.haddock_restraints.haddock3_restrain_bodies Version: 5.0.1
2025-05-12 17:05:57,619 [MainThread  ] [INFO ]  /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_18387556-964c-47f9-844b-69f1382659de directory successfully created
2025-05-12 17:05:57,620 [MainThread  ] [INFO ]  Copy: ./data/antibody//4G6K_clean.pdb to /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_18387556-964c-47f9-844b-69f1382659de
2025-05-12 17:05:57,620 [MainThread  ] [INFO ]  haddock3-restraints restrain_bodies /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_18387556-964c-47f9-844b-69f1382659de/4G6K_clean.pdb &> /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_18387556-964c-47f9-844b-69f1382659de/antibody-unambig.tbl

2025-05-12 17:05:57,986 [MainThread  ] [INFO ]  Executing: haddock3-restraints restrain_bodies /home/rchave

0

## Docking

In [36]:
# repeat variables to run the code above
ab_pdb_clean = f'{out_path}{ab_id}_clean.pdb'
ag_pdb_clean = f'{out_path}{ag_id}_clean.pdb'
ref_pdb_clean = f'{out_path}{ref_id}_clean.pdb'
complex_tbl = f'{out_path}ambig-paratope-NMR-epitope.tbl'
body_tbl = f'{out_path}antibody-unambig.tbl'

### Create topology

In [37]:
from biobb_haddock.haddock.topology import topology
step_idx = 0
mol1_output_top_zip_path = f'{out_path}/docking/{step_idx}/top_mol1.zip'
mol2_output_top_zip_path = f'{out_path}/docking/{step_idx}/top_mol2.zip'
wf_topology              = f'{out_path}/docking/{step_idx}/wf_topology.zip'

topology(mol1_input_pdb_path        = ab_pdb_clean,
         mol2_input_pdb_path        = ag_pdb_clean,
         mol1_output_top_zip_path   = mol1_output_top_zip_path,
         mol2_output_top_zip_path   = mol2_output_top_zip_path,
         output_haddock_wf_data_zip = wf_topology,
         properties                 = def_dict())

2025-05-12 17:05:58,041 [MainThread  ] [INFO ]  Module: biobb_haddock.haddock.topology Version: 5.0.1
2025-05-12 17:05:58,042 [MainThread  ] [INFO ]  /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_152434f7-e78f-4b7c-acba-cccce84a21e5 directory successfully created
2025-05-12 17:05:58,043 [MainThread  ] [INFO ]  Copy: ./data/antibody/4G6K_clean.pdb to /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_152434f7-e78f-4b7c-acba-cccce84a21e5
2025-05-12 17:05:58,044 [MainThread  ] [INFO ]  Copy: ./data/antibody/4I1B_clean.pdb to /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_152434f7-e78f-4b7c-acba-cccce84a21e5
2025-05-12 17:05:58,045 [MainThread  ] [INFO ]  haddock3 da80d7fd-e147-4b37-9ac4-97fb2af9693b/haddock.cfg

2025-05-12 17:06:00,072 [MainThread  ] [INFO ]  Executing: haddock3 da80d7fd-e147-4b37-9ac4-97fb2af9693b/haddock.cfg...
2025-05-12 17:06:00,073 [MainThread  ] [INFO ]  Exit code: 0
2

0

### Rigid Body sampling

In [38]:
from biobb_haddock.haddock.rigid_body import rigid_body

properties={
    'cfg': {
        'tolerance': 2,
        'sampling': 10, # subir
    }
}

step_idx = 1
docking_output_zip_path = f'{out_path}docking/{step_idx}/docking.zip'
wf_rigidbody            = f'{out_path}docking/{step_idx}/wf_rigidbody.zip'

rigid_body(input_haddock_wf_data_zip     = wf_topology,
           docking_output_zip_path       = docking_output_zip_path,
           ambig_restraints_table_path   = complex_tbl,
           unambig_restraints_table_path = body_tbl,
           output_haddock_wf_data_zip    = wf_rigidbody,
           properties                    = def_dict(properties))

2025-05-12 17:06:00,120 [MainThread  ] [INFO ]  Module: biobb_haddock.haddock.rigid_body Version: 5.0.1
2025-05-12 17:06:00,121 [MainThread  ] [INFO ]  /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_f68eeec1-b87d-4132-8196-80821674fd4a directory successfully created
2025-05-12 17:06:00,122 [MainThread  ] [INFO ]  Copy: ./data/antibody/ambig-paratope-NMR-epitope.tbl to /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_f68eeec1-b87d-4132-8196-80821674fd4a
2025-05-12 17:06:00,122 [MainThread  ] [INFO ]  Copy: ./data/antibody/antibody-unambig.tbl to /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_f68eeec1-b87d-4132-8196-80821674fd4a
2025-05-12 17:06:00,125 [MainThread  ] [INFO ]  Extracting: /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/data/antibody/docking/0/wf_topology.zip
2025-05-12 17:06:00,125 [MainThread  ] [INFO ]  to:
2025-05-12 17:06:00,126 [MainThread  ] [

0

### CAPRI evaluation

In [39]:
from biobb_haddock.haddock.capri_eval import capri_eval

step_idx = 2
output_evaluation_zip_path = f'{out_path}docking/{step_idx}/caprieval.zip'
wf_caprieval               = f'{out_path}docking/{step_idx}/wf_caprieval.zip'

capri_eval(input_haddock_wf_data_zip  = wf_rigidbody,
           reference_pdb_path         = ref_pdb_clean,
           output_evaluation_zip_path = output_evaluation_zip_path,
           output_haddock_wf_data_zip = wf_caprieval,
           properties                 = def_dict())

2025-05-12 17:06:32,354 [MainThread  ] [INFO ]  Module: biobb_haddock.haddock.capri_eval Version: 5.0.1
2025-05-12 17:06:32,356 [MainThread  ] [INFO ]  /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_0d7e617a-607a-4a95-9904-e6284643de7f directory successfully created
2025-05-12 17:06:32,358 [MainThread  ] [INFO ]  Copy: ./data/antibody/4G6M_clean.pdb to /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_0d7e617a-607a-4a95-9904-e6284643de7f
2025-05-12 17:06:32,386 [MainThread  ] [INFO ]  Extracting: /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/data/antibody/docking/1/wf_rigidbody.zip
2025-05-12 17:06:32,387 [MainThread  ] [INFO ]  to:
2025-05-12 17:06:32,387 [MainThread  ] [INFO ]  ['386a2c50-188b-4d94-a2cd-6447c07afcf3/0_topoaa', '386a2c50-188b-4d94-a2cd-6447c07afcf3/1_rigidbody', '386a2c50-188b-4d94-a2cd-6447c07afcf3/analysis', '386a2c50-188b-4d94-a2cd-6447c07afcf3/data', '386a2c50-188b-4d94-a2c

0

### Select Top structures

In [40]:
from biobb_haddock.haddock.sele_top import sele_top

properties={
    'cfg': {
        'select': 8,
    }
}

step_idx = 3
output_selection_zip_path = f'{out_path}docking/{step_idx}/selected.zip'
wf_seletop                = f'{out_path}docking/{step_idx}/wf_seletop.zip'

sele_top(input_haddock_wf_data_zip  = wf_caprieval,
         output_selection_zip_path  = output_selection_zip_path,
         output_haddock_wf_data_zip = wf_seletop,
         properties                 = def_dict(properties))

2025-05-12 17:06:35,518 [MainThread  ] [INFO ]  Module: biobb_haddock.haddock.sele_top Version: 5.0.1
2025-05-12 17:06:35,519 [MainThread  ] [INFO ]  /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_dd412db5-3fcd-4eb4-b938-816d542d8a95 directory successfully created
2025-05-12 17:06:35,545 [MainThread  ] [INFO ]  Extracting: /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/data/antibody/docking/2/wf_caprieval.zip
2025-05-12 17:06:35,545 [MainThread  ] [INFO ]  to:
2025-05-12 17:06:35,546 [MainThread  ] [INFO ]  ['1f0a2478-93d8-4594-80e7-9c35ab7ad5e5/0_topoaa', '1f0a2478-93d8-4594-80e7-9c35ab7ad5e5/1_rigidbody', '1f0a2478-93d8-4594-80e7-9c35ab7ad5e5/2_caprieval', '1f0a2478-93d8-4594-80e7-9c35ab7ad5e5/analysis', '1f0a2478-93d8-4594-80e7-9c35ab7ad5e5/data', '1f0a2478-93d8-4594-80e7-9c35ab7ad5e5/traceback', '1f0a2478-93d8-4594-80e7-9c35ab7ad5e5/log', '1f0a2478-93d8-4594-80e7-9c35ab7ad5e5/traceback/consensus.tsv', '1f0a2478-93d8-4594-8

2025-05-12 17:06:35,564 [MainThread  ] [INFO ]  1f0a2478-93d8-4594-80e7-9c35ab7ad5e5/analysis/1_rigidbody_analysis/irmsd_desolv.html
2025-05-12 17:06:35,564 [MainThread  ] [INFO ]  1f0a2478-93d8-4594-80e7-9c35ab7ad5e5/analysis/1_rigidbody_analysis/lrmsd_vdw.html
2025-05-12 17:06:35,564 [MainThread  ] [INFO ]  1f0a2478-93d8-4594-80e7-9c35ab7ad5e5/analysis/1_rigidbody_analysis/ilrmsd_clt.html
2025-05-12 17:06:35,564 [MainThread  ] [INFO ]  1f0a2478-93d8-4594-80e7-9c35ab7ad5e5/analysis/1_rigidbody_analysis/ilrmsd_desolv.html
2025-05-12 17:06:35,565 [MainThread  ] [INFO ]  1f0a2478-93d8-4594-80e7-9c35ab7ad5e5/analysis/1_rigidbody_analysis/ilrmsd_air.html
2025-05-12 17:06:35,565 [MainThread  ] [INFO ]  1f0a2478-93d8-4594-80e7-9c35ab7ad5e5/analysis/1_rigidbody_analysis/summary.tgz
2025-05-12 17:06:35,566 [MainThread  ] [INFO ]  1f0a2478-93d8-4594-80e7-9c35ab7ad5e5/analysis/1_rigidbody_analysis/lrmsd_desolv.html
2025-05-12 17:06:35,566 [MainThread  ] [INFO ]  1f0a2478-93d8-4594-80e7-9c35ab7ad

0

### 2nd CAPRI evalutation

In [41]:
from biobb_haddock.haddock.capri_eval import capri_eval

step_idx = 4
output_evaluation_zip_path2 = f'{out_path}docking/{step_idx}/caprieval2.zip'
wf_caprieval2               = f'{out_path}docking/{step_idx}/wf_caprieval2.zip'

capri_eval(input_haddock_wf_data_zip  = wf_seletop,
           reference_pdb_path         = ref_pdb_clean,
           output_evaluation_zip_path = output_evaluation_zip_path2,
           output_haddock_wf_data_zip = wf_caprieval2,
           properties                 = def_dict())

2025-05-12 17:06:39,018 [MainThread  ] [INFO ]  Module: biobb_haddock.haddock.capri_eval Version: 5.0.1
2025-05-12 17:06:39,019 [MainThread  ] [INFO ]  /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_85d3f5f8-ed8a-4c74-9393-2980874e38bb directory successfully created
2025-05-12 17:06:39,020 [MainThread  ] [INFO ]  Copy: ./data/antibody/4G6M_clean.pdb to /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_85d3f5f8-ed8a-4c74-9393-2980874e38bb


2025-05-12 17:06:39,050 [MainThread  ] [INFO ]  Extracting: /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/data/antibody/docking/3/wf_seletop.zip
2025-05-12 17:06:39,050 [MainThread  ] [INFO ]  to:
2025-05-12 17:06:39,050 [MainThread  ] [INFO ]  ['97c1b43e-e6c1-4c3d-9d23-1a5584e8e43a/0_topoaa', '97c1b43e-e6c1-4c3d-9d23-1a5584e8e43a/1_rigidbody', '97c1b43e-e6c1-4c3d-9d23-1a5584e8e43a/2_caprieval', '97c1b43e-e6c1-4c3d-9d23-1a5584e8e43a/3_seletop', '97c1b43e-e6c1-4c3d-9d23-1a5584e8e43a/analysis', '97c1b43e-e6c1-4c3d-9d23-1a5584e8e43a/data', '97c1b43e-e6c1-4c3d-9d23-1a5584e8e43a/traceback', '97c1b43e-e6c1-4c3d-9d23-1a5584e8e43a/log', '97c1b43e-e6c1-4c3d-9d23-1a5584e8e43a/traceback/consensus.tsv', '97c1b43e-e6c1-4c3d-9d23-1a5584e8e43a/traceback/traceback.tsv', '97c1b43e-e6c1-4c3d-9d23-1a5584e8e43a/traceback/traceback.html', '97c1b43e-e6c1-4c3d-9d23-1a5584e8e43a/analysis/1_rigidbody_analysis', '97c1b43e-e6c1-4c3d-9d23-1a5584e8e43a/analysis/2_caprieval_analysis', '97

0

### Flexible Refinement

In [42]:
from biobb_haddock.haddock.flex_ref import flex_ref

step_idx = 5
refinement_output_zip_path = f'{out_path}docking/{step_idx}/flexref.zip'
wf_flexref                 = f'{out_path}docking/{step_idx}/wf_flexref.zip'

flex_ref(input_haddock_wf_data_zip     = wf_caprieval2,
         refinement_output_zip_path    = refinement_output_zip_path,
         ambig_restraints_table_path   = complex_tbl,
         unambig_restraints_table_path = body_tbl,
         output_haddock_wf_data_zip    = wf_flexref,
         properties                    = def_dict())

2025-05-12 17:06:41,711 [MainThread  ] [INFO ]  Module: biobb_haddock.haddock.flex_ref Version: 5.0.1
2025-05-12 17:06:41,711 [MainThread  ] [INFO ]  /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_983364df-038f-4530-a225-84a2ad7828df directory successfully created
2025-05-12 17:06:41,712 [MainThread  ] [INFO ]  Copy: ./data/antibody/ambig-paratope-NMR-epitope.tbl to /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_983364df-038f-4530-a225-84a2ad7828df
2025-05-12 17:06:41,713 [MainThread  ] [INFO ]  Copy: ./data/antibody/antibody-unambig.tbl to /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_983364df-038f-4530-a225-84a2ad7828df
2025-05-12 17:06:41,745 [MainThread  ] [INFO ]  Extracting: /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/data/antibody/docking/4/wf_caprieval2.zip
2025-05-12 17:06:41,745 [MainThread  ] [INFO ]  to:
2025-05-12 17:06:41,746 [MainThread  ] [

0

### 3nd CAPRI evalutation

In [43]:
from biobb_haddock.haddock.capri_eval import capri_eval

step_idx = 6
output_evaluation_zip_path3 = f'{out_path}docking/{step_idx}/caprieval3.zip'
wf_caprieval3               = f'{out_path}docking/{step_idx}/wf_caprieval3.zip'

capri_eval(input_haddock_wf_data_zip  = wf_flexref,
           reference_pdb_path         = ref_pdb_clean,
           output_evaluation_zip_path = output_evaluation_zip_path3,
           output_haddock_wf_data_zip = wf_caprieval3,
           properties                 = def_dict())

2025-05-12 17:09:00,202 [MainThread  ] [INFO ]  Module: biobb_haddock.haddock.capri_eval Version: 5.0.1
2025-05-12 17:09:00,203 [MainThread  ] [INFO ]  /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_43920ab1-eace-4568-a50a-1d951c11f994 directory successfully created
2025-05-12 17:09:00,204 [MainThread  ] [INFO ]  Copy: ./data/antibody/4G6M_clean.pdb to /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_43920ab1-eace-4568-a50a-1d951c11f994
2025-05-12 17:09:00,248 [MainThread  ] [INFO ]  Extracting: /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/data/antibody/docking/5/wf_flexref.zip
2025-05-12 17:09:00,248 [MainThread  ] [INFO ]  to:
2025-05-12 17:09:00,249 [MainThread  ] [INFO ]  ['bfaaacb6-e077-4a13-ace7-6f6c546aa865/0_topoaa', 'bfaaacb6-e077-4a13-ace7-6f6c546aa865/1_rigidbody', 'bfaaacb6-e077-4a13-ace7-6f6c546aa865/2_caprieval', 'bfaaacb6-e077-4a13-ace7-6f6c546aa865/3_seletop', 'bfaaacb6-e077-4a

0

### Energy Minimization Refinement

In [44]:
from biobb_haddock.haddock.em_ref import em_ref

step_idx = 7
refinement_output_zip_path = f'{out_path}docking/{step_idx}/emref.zip'
wf_emref                   = f'{out_path}docking/{step_idx}/wf_emref.zip'

em_ref(input_haddock_wf_data_zip  = wf_caprieval3,
       refinement_output_zip_path = refinement_output_zip_path,
       ambig_restraints_table_path   = complex_tbl,
       unambig_restraints_table_path = body_tbl,
       output_haddock_wf_data_zip = wf_emref,
       properties                 = def_dict())

2025-05-12 17:09:03,297 [MainThread  ] [INFO ]  Module: biobb_haddock.haddock.em_ref Version: 5.0.1
2025-05-12 17:09:03,298 [MainThread  ] [INFO ]  /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_f9e6ed4b-956c-44b9-b689-ae817680d4c7 directory successfully created
2025-05-12 17:09:03,298 [MainThread  ] [INFO ]  Copy: ./data/antibody/ambig-paratope-NMR-epitope.tbl to /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_f9e6ed4b-956c-44b9-b689-ae817680d4c7
2025-05-12 17:09:03,299 [MainThread  ] [INFO ]  Copy: ./data/antibody/antibody-unambig.tbl to /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_f9e6ed4b-956c-44b9-b689-ae817680d4c7
2025-05-12 17:09:03,348 [MainThread  ] [INFO ]  Extracting: /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/data/antibody/docking/6/wf_caprieval3.zip
2025-05-12 17:09:03,348 [MainThread  ] [INFO ]  to:
2025-05-12 17:09:03,349 [MainThread  ] [IN

0

### 4rd CAPRI evaluation

In [45]:
from biobb_haddock.haddock.capri_eval import capri_eval

step_idx = 8
output_evaluation_zip_path4 = f'{out_path}docking/{step_idx}/caprieval4.zip'
wf_caprieval4               = f'{out_path}docking/{step_idx}/wf_caprieval4.zip'

capri_eval(input_haddock_wf_data_zip  = wf_emref,
           reference_pdb_path         = ref_pdb_clean,
           output_evaluation_zip_path = output_evaluation_zip_path4,
           output_haddock_wf_data_zip = wf_caprieval4,
           properties                 = def_dict())

2025-05-12 17:09:17,350 [MainThread  ] [INFO ]  Module: biobb_haddock.haddock.capri_eval Version: 5.0.1
2025-05-12 17:09:17,350 [MainThread  ] [INFO ]  /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_a937640b-0f40-4005-8b06-ab68c2c03883 directory successfully created
2025-05-12 17:09:17,351 [MainThread  ] [INFO ]  Copy: ./data/antibody/4G6M_clean.pdb to /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_a937640b-0f40-4005-8b06-ab68c2c03883
2025-05-12 17:09:17,414 [MainThread  ] [INFO ]  Extracting: /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/data/antibody/docking/7/wf_emref.zip
2025-05-12 17:09:17,415 [MainThread  ] [INFO ]  to:
2025-05-12 17:09:17,415 [MainThread  ] [INFO ]  ['22eafafd-ad89-45e3-8cd5-1800aab2cb97/0_topoaa', '22eafafd-ad89-45e3-8cd5-1800aab2cb97/1_rigidbody', '22eafafd-ad89-45e3-8cd5-1800aab2cb97/2_caprieval', '22eafafd-ad89-45e3-8cd5-1800aab2cb97/3_seletop', '22eafafd-ad89-45e3

0

### Clustering using FCC

In [46]:
from biobb_haddock.haddock.clust_fcc import clust_fcc

step_idx = 9
output_cluster_zip_path = f'{out_path}docking/{step_idx}/clustfcc.zip'
wf_clustfcc             = f'{out_path}docking/{step_idx}/wf_clustfcc.zip'

clust_fcc(input_haddock_wf_data_zip = wf_caprieval4,
         output_cluster_zip_path    = output_cluster_zip_path,
         output_haddock_wf_data_zip = wf_clustfcc,
         properties                 = def_dict())

2025-05-12 17:09:20,341 [MainThread  ] [INFO ]  Module: biobb_haddock.haddock.clust_fcc Version: 5.0.1
2025-05-12 17:09:20,341 [MainThread  ] [INFO ]  /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_a5a3e9f9-56f1-4495-8b4b-84ce63a91fa0 directory successfully created
2025-05-12 17:09:20,409 [MainThread  ] [INFO ]  Extracting: /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/data/antibody/docking/8/wf_caprieval4.zip
2025-05-12 17:09:20,410 [MainThread  ] [INFO ]  to:
2025-05-12 17:09:20,411 [MainThread  ] [INFO ]  ['57ea5b2a-018b-4d95-a2d1-30e62548d7f4/0_topoaa', '57ea5b2a-018b-4d95-a2d1-30e62548d7f4/1_rigidbody', '57ea5b2a-018b-4d95-a2d1-30e62548d7f4/2_caprieval', '57ea5b2a-018b-4d95-a2d1-30e62548d7f4/3_seletop', '57ea5b2a-018b-4d95-a2d1-30e62548d7f4/4_caprieval', '57ea5b2a-018b-4d95-a2d1-30e62548d7f4/5_flexref', '57ea5b2a-018b-4d95-a2d1-30e62548d7f4/6_caprieval', '57ea5b2a-018b-4d95-a2d1-30e62548d7f4/7_emref', '57ea5b2a-018b-4d95

0

### Selecting top clusters

In [47]:
from biobb_haddock.haddock.sele_top_clusts import sele_top_clusts

properties={
    'cfg': {
        'top_models': 4,
    },
}

step_idx = 10
output_seletopclusts_zip_path = f'{out_path}docking/{step_idx}/seletopclusts.zip'
wf_seletopclusts              = f'{out_path}docking/{step_idx}/wf_seletopclusts.zip'

sele_top_clusts(input_haddock_wf_data_zip  = wf_clustfcc,
                output_selection_zip_path  = output_seletopclusts_zip_path,
                output_haddock_wf_data_zip = wf_seletopclusts,
                properties                 = def_dict(properties))

2025-05-12 17:09:23,078 [MainThread  ] [INFO ]  Module: biobb_haddock.haddock.sele_top_clusts Version: 5.0.1
2025-05-12 17:09:23,078 [MainThread  ] [INFO ]  /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_4a38d41c-65ce-4f26-ac32-2ebee05a25e6 directory successfully created
2025-05-12 17:09:23,154 [MainThread  ] [INFO ]  Extracting: /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/data/antibody/docking/9/wf_clustfcc.zip
2025-05-12 17:09:23,155 [MainThread  ] [INFO ]  to:
2025-05-12 17:09:23,155 [MainThread  ] [INFO ]  ['7979d8c1-a94e-45a5-8e4e-a00c821adea8/0_topoaa', '7979d8c1-a94e-45a5-8e4e-a00c821adea8/1_rigidbody', '7979d8c1-a94e-45a5-8e4e-a00c821adea8/2_caprieval', '7979d8c1-a94e-45a5-8e4e-a00c821adea8/3_seletop', '7979d8c1-a94e-45a5-8e4e-a00c821adea8/4_caprieval', '7979d8c1-a94e-45a5-8e4e-a00c821adea8/5_flexref', '7979d8c1-a94e-45a5-8e4e-a00c821adea8/6_caprieval', '7979d8c1-a94e-45a5-8e4e-a00c821adea8/7_emref', '7979d8c1-a94e-

0

### 5th CAPRI evaluation

In [48]:
from biobb_haddock.haddock.capri_eval import capri_eval

step_idx = 11
output_evaluation_zip_path5 = f'{out_path}docking/{step_idx}/caprieval5.zip'
wf_caprieval5               = f'{out_path}docking/{step_idx}/wf_caprieval5.zip'

capri_eval(input_haddock_wf_data_zip  = wf_seletopclusts,
           reference_pdb_path         = ref_pdb_clean,
           output_evaluation_zip_path = output_evaluation_zip_path5,
           output_haddock_wf_data_zip = wf_caprieval5,
           properties                 = def_dict())

2025-05-12 17:09:37,274 [MainThread  ] [INFO ]  Module: biobb_haddock.haddock.capri_eval Version: 5.0.1
2025-05-12 17:09:37,275 [MainThread  ] [INFO ]  /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_0d3a57ad-6ee4-4d28-a277-4e26621f6b0a directory successfully created
2025-05-12 17:09:37,275 [MainThread  ] [INFO ]  Copy: ./data/antibody/4G6M_clean.pdb to /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_0d3a57ad-6ee4-4d28-a277-4e26621f6b0a
2025-05-12 17:09:37,386 [MainThread  ] [INFO ]  Extracting: /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/data/antibody/docking/10/wf_seletopclusts.zip
2025-05-12 17:09:37,386 [MainThread  ] [INFO ]  to:
2025-05-12 17:09:37,386 [MainThread  ] [INFO ]  ['c0aa3eb5-0d35-4ad7-a549-b73698ababe0/00_topoaa', 'c0aa3eb5-0d35-4ad7-a549-b73698ababe0/01_rigidbody', 'c0aa3eb5-0d35-4ad7-a549-b73698ababe0/02_caprieval', 'c0aa3eb5-0d35-4ad7-a549-b73698ababe0/03_seletop', 'c0aa3

0

### Contacts analysis

In [49]:
from biobb_haddock.haddock.contact_map import contact_map

step_idx = 12
output_contactmap_zip_path = f'{out_path}docking/{step_idx}/contact_map.zip'
wf_contact_map             = f'{out_path}docking/{step_idx}/wf_contact_map.zip'

contact_map(input_haddock_wf_data_zip  = wf_caprieval5,
            output_contactmap_zip_path = output_contactmap_zip_path,
            output_haddock_wf_data_zip = wf_contact_map,
            properties                 = def_dict())

2025-05-12 17:09:40,752 [MainThread  ] [INFO ]  Module: biobb_haddock.haddock.contact_map Version: 5.0.1
2025-05-12 17:09:40,753 [MainThread  ] [INFO ]  /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_043a2f7e-f417-4ea4-8199-d3471d6863e6 directory successfully created
2025-05-12 17:09:40,762 [MainThread  ] [INFO ]  Copy: ./data/antibody/docking/11/wf_caprieval5.zip to /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_043a2f7e-f417-4ea4-8199-d3471d6863e6
2025-05-12 17:09:40,878 [MainThread  ] [INFO ]  Extracting: /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/data/antibody/docking/11/wf_caprieval5.zip
2025-05-12 17:09:40,879 [MainThread  ] [INFO ]  to:
2025-05-12 17:09:40,879 [MainThread  ] [INFO ]  ['f7578457-3e15-45b6-b025-86a4056d318b/00_topoaa', 'f7578457-3e15-45b6-b025-86a4056d318b/01_rigidbody', 'f7578457-3e15-45b6-b025-86a4056d318b/02_caprieval', 'f7578457-3e15-45b6-b025-86a4056d318b/03_sele

0

## Results

In [3]:
step_idx = 12
output_contactmap_zip_path = f'{out_path}docking/{step_idx}/contact_map.zip'
wf_contact_map             = f'{out_path}docking/{step_idx}/wf_contact_map.zip'


with zipfile.ZipFile(wf_contact_map, 'r') as zip_ref:
    zip_ref.extractall(out_path+'/final_results')

In [28]:
webbrowser.open("http://localhost:8888/edit/biobb_wf_haddock/notebooks/data/antibody/final_results/analysis/12_contactmap_analysis/report.html")
#!python3 -m http.server

True

In [27]:
tsv_dir = output_haddock_wf_data_zip[:-4]+'/8_caprieval/'
# Load the cluster and single data into pandas DataFrames
cluster_df = pd.read_csv(tsv_dir + 'capri_clt.tsv', sep='\t',comment='#')
single_df = pd.read_csv(tsv_dir + 'capri_ss.tsv', sep='\t',comment='#')

# DockQ: incorrect (<0.23), acceptable (0.23-0.49), medium (0.49-0.80), and high (>=0.80) 
display(single_df.head())
single_df = single_df.sort_values(by='dockq', ascending=False)
display(single_df.head())
display(cluster_df.head())

,model,md5,caprieval_rank,score,irmsd,fnat,lrmsd,ilrmsd,dockq,rmsd,...,dihe,elec,improper,rdcs,rg,sym,total,vdw,vean,xpcs
0,../7_emref/emref_7.pdb,-,1,-117.701,12.814,0.073,18.964,18.776,0.085,11.950,...,0.0,-286.254,0.0,0.0,0.0,0.0,-332.036,-47.892,0.0,0.0
1,../7_emref/emref_1.pdb,-,2,-103.956,10.943,0.073,18.897,18.034,0.086,10.878,...,0.0,-226.571,0.0,0.0,0.0,0.0,-250.697,-38.348,0.0,0.0
2,../7_emref/emref_2.pdb,-,3,-87.783,12.370,0.055,20.782,19.740,0.071,12.726,...,0.0,-222.792,0.0,0.0,0.0,0.0,-250.085,-36.871,0.0,0.0
3,../7_emref/emref_5.pdb,-,4,-83.329,6.200,0.073,10.348,11.039,0.177,5.858,...,0.0,-360.748,0.0,0.0,0.0,0.0,-376.941,-18.320,0.0,0.0
4,../7_emref/emref_3.pdb,-,5,-81.900,12.479,0.091,18.850,18.578,0.091,11.998,...,0.0,-252.467,0.0,0.0,0.0,0.0,-276.038,-29.830,0.0,0.0


,model,md5,caprieval_rank,score,irmsd,fnat,lrmsd,ilrmsd,dockq,rmsd,...,dihe,elec,improper,rdcs,rg,sym,total,vdw,vean,xpcs
6,../7_emref/emref_6.pdb,-,7,-78.496,3.782,0.145,6.579,7.105,0.302,3.413,...,0.0,-302.977,0.0,0.0,0.0,0.0,-298.292,-20.686,0.0,0.0
3,../7_emref/emref_5.pdb,-,4,-83.329,6.200,0.073,10.348,11.039,0.177,5.858,...,0.0,-360.748,0.0,0.0,0.0,0.0,-376.941,-18.320,0.0,0.0
5,../7_emref/emref_8.pdb,-,6,-81.642,7.256,0.127,14.158,12.438,0.144,7.550,...,0.0,-319.729,0.0,0.0,0.0,0.0,-317.847,-21.555,0.0,0.0
4,../7_emref/emref_3.pdb,-,5,-81.900,12.479,0.091,18.850,18.578,0.091,11.998,...,0.0,-252.467,0.0,0.0,0.0,0.0,-276.038,-29.830,0.0,0.0
1,../7_emref/emref_1.pdb,-,2,-103.956,10.943,0.073,18.897,18.034,0.086,10.878,...,0.0,-226.571,0.0,0.0,0.0,0.0,-250.697,-38.348,0.0,0.0


,cluster_rank,cluster_id,n,under_eval,score,score_std,irmsd,irmsd_std,fnat,fnat_std,...,bsa_std,desolv,desolv_std,elec,elec_std,total,total_std,vdw,vdw_std,caprieval_rank
0,-,-,10,-,-80.373,20.439,12.036,0.633,0.068,0.015,...,178.9,-6.904,9.385,-227.175,16.368,-241.623,31.77,-29.544,10.011,1


# Barnase-Barstar protein complex

In [2]:
# Barnase-Barstar protein complex
# From Chen, R., Mintseris, J., Janin, J. and Weng, Z. (2003)
# A protein–protein docking benchmark. 
# Proteins, 52: 88-91. https://doi-org.sire.ub.edu/10.1002/prot.10390
barnase_id = "1A2P"
barnase_ch = "B"
barstar_id = "1A19"
barstar_ch = "A"
complex_id = "1BRS" # barnase_barstar_complex
complex_ch = "A,D"
out_path = 'data/barnase_barstar/'
os.makedirs(out_path, exist_ok=True)

## Prepare pdbs

In [9]:
# Downloading desired PDB files
# Import module
from biobb_io.api.pdb import pdb

# Create properties dict and inputs/outputs
barnase_pdb = f'{out_path}{barnase_id}.pdb'
barstar_pdb = f'{out_path}{barstar_id}.pdb'
complex_pdb = f'{out_path}{complex_id}.pdb'

# Create and launch bb
pdb(output_pdb_path=barnase_pdb, properties=def_dict({'pdb_code': barnase_id}))
pdb(output_pdb_path=barstar_pdb, properties=def_dict({'pdb_code': barstar_id}))
pdb(output_pdb_path=complex_pdb, properties=def_dict({'pdb_code': complex_id}))

0

In [10]:
# These are the pdbs we get from RCSB
show_pdbs([barnase_pdb, barstar_pdb, complex_pdb])

In [11]:
# Filtering specific chains: we need to get rid of repeated chains
from biobb_pdb_tools.pdb_tools.biobb_pdb_selchain import biobb_pdb_selchain

# Create properties dict and inputs/outputs
barnase_pdb_ch = f'{out_path}{barnase_id}_ch.pdb'
barstar_pdb_ch = f'{out_path}{barstar_id}_ch.pdb'
complex_pdb_ch = f'{out_path}{complex_id}_ch.pdb'

# # Create and launch bb
biobb_pdb_selchain(input_file_path  = barnase_pdb,
                   output_file_path = barnase_pdb_ch,
                   properties       = def_dict({'chains': barnase_ch}))

biobb_pdb_selchain(input_file_path  = barstar_pdb,
                   output_file_path = barstar_pdb_ch,
                   properties       = def_dict({'chains': barstar_ch}))

biobb_pdb_selchain(input_file_path  = complex_pdb,
                   output_file_path = complex_pdb_ch,
                   properties       = def_dict({'chains': complex_ch}))

0

In [12]:
# On a real case we don't have the reference to know how the proteins bind each other
# What information can use to guide the process?
show_pdbs([barnase_pdb_ch, barstar_pdb_ch, complex_pdb_ch])

## Prepare AIRs

In [13]:
# Solvent accessibility: 
from biobb_haddock.haddock_restraints.haddock3_accessibility import haddock3_accessibility

# Create properties dict and inputs/outputs
barnase_sasa_out = f'{out_path}{barnase_id}_sasa_out.txt'
barstar_sasa_out = f'{out_path}{barstar_id}_sasa_out.txt'
barnase_sasa_actpass = f'{out_path}{barnase_id}_sasa_actpass.txt'
barstar_sasa_actpass = f'{out_path}{barstar_id}_sasa_actpass.txt'

cutoff = 0.3
# Barnase Chain
haddock3_accessibility(
        input_pdb_path            = barnase_pdb_ch,
        output_accessibility_path = barnase_sasa_out,
        output_actpass_path       = barnase_sasa_actpass,
        properties                = def_dict({'chain': barnase_ch,
                                              'cutoff': cutoff}))
# Barstar Chain
haddock3_accessibility(
        input_pdb_path            = barstar_pdb_ch,
        output_accessibility_path = barstar_sasa_out,
        output_actpass_path       = barstar_sasa_actpass,
        properties                = def_dict({'chain': barstar_ch,
                                              'cutoff': cutoff}))

0

In [14]:
# Careful! Pockets are good places to bind but have low accessibility
view1 = display_actpass(barnase_pdb_ch, barnase_sasa_actpass)
view2 = display_actpass(barstar_pdb_ch, barstar_sasa_actpass)
ipywidgets.HBox([view1, view2])

In [15]:
# Electrostatic energies:
# We see a postive charge in the binding site of barnase and a negative charge in the binding site of barstar
show_pdbs([barnase_pdb_ch, barstar_pdb_ch],surface=True)

In [16]:
# Obtain passive from active selection
from biobb_haddock.haddock_restraints.haddock3_passive_from_active import haddock3_passive_from_active

barnase_pass2act = f'{out_path}{barnase_id}_manual_actpass.txt'
barstar_pass2act = f'{out_path}{barstar_id}_manual_actpass.txt'

haddock3_passive_from_active( 
    input_pdb_path      = barnase_pdb_ch,
    output_actpass_path = barnase_pass2act,
    properties          = def_dict({'active_list' : '27,73,83,87'})
)

haddock3_passive_from_active( 
    input_pdb_path      = barstar_pdb_ch,
    output_actpass_path = barstar_pass2act,
    properties          = def_dict({'active_list' : '33,35,39,43'})
)

0

In [17]:
view1 = display_actpass(barnase_pdb_ch, barnase_pass2act, opacity=0.3)
view2 = display_actpass(barstar_pdb_ch, barstar_pass2act, opacity=0.3)
ipywidgets.HBox([view1, view2])

In [18]:
# Convert active/passive to ambiguous restraints
from biobb_haddock.haddock_restraints.haddock3_actpass_to_ambig import haddock3_actpass_to_ambig

# With SASA
barnase_barstar_sasa_tbl = f'{out_path}barnase_barstar_sasa.tbl'
haddock3_actpass_to_ambig( 
    input_actpass1_path = barnase_sasa_actpass,
    input_actpass2_path = barstar_sasa_actpass,    
    output_tbl_path     = barnase_barstar_sasa_tbl,
    properties          = def_dict({'pass_to_act' : True,  # tbl need actives, we use the passive as active
                                    'segid_one': barnase_ch, 
                                    'segid_two': barstar_ch}))

# With manual active/passive
barnase_barstar_manual_tbl = f'{out_path}barnase_barstar_manual.tbl'
haddock3_actpass_to_ambig( 
    input_actpass1_path = barnase_pass2act,
    input_actpass2_path = barstar_pass2act,    
    output_tbl_path     = barnase_barstar_manual_tbl,
    properties          = def_dict({'segid_one': barnase_ch,
                                    'segid_two': barstar_ch}))

# The restrain have the next format:
# assign (selection1) (selection2) distance, lower-bound correction, upper-bound correction

0

In [19]:
# Validate tbl
!haddock3-restraints validate_tbl {barnase_barstar_sasa_tbl} --silent
!haddock3-restraints validate_tbl {barnase_barstar_manual_tbl} --silent

## Docking

In [3]:
barnase_pdb_ch = f'{out_path}{barnase_id}_ch.pdb'
barstar_pdb_ch = f'{out_path}{barstar_id}_ch.pdb'
complex_pdb_ch = f'{out_path}{complex_id}_ch.pdb'
barnase_barstar_manual_tbl = 'data/barnase_barstar/barnase_barstar_manual.tbl'

### 0. Topology

In [4]:
from biobb_haddock.haddock.topology import topology

properties=def_dict({
    'cfg': {
        'tolerance': 0,
    },
})

step_idx = 0
barnase_top_zip_path = f'{out_path}{step_idx}/barnase_top.zip'
barstar_top_zip_path = f'{out_path}{step_idx}/barstar_top.zip'
wf_topology          = f'{out_path}{step_idx}/wf_topology.zip'

topology(mol1_input_pdb_path        = barnase_pdb_ch,
         mol2_input_pdb_path        = barstar_pdb_ch,
         mol1_output_top_zip_path   = barnase_top_zip_path,
         mol2_output_top_zip_path   = barstar_top_zip_path,
         output_haddock_wf_data_zip = wf_topology,
         properties                 = properties)

0

### 1. Rigid body docking

In [5]:
from biobb_haddock.haddock.rigid_body import rigid_body

properties=def_dict({
    'cfg': {
        'tolerance': 5,
        'sampling': 10,
        # turn on random definiton of AIRs
        'ranair': False
    },
})

step_idx = 1
docking_output_zip_path = f'{out_path}{step_idx}/docking.zip'
wf_rigidbody            = f'{out_path}{step_idx}/wf_rigidbody.zip'

rigid_body(input_haddock_wf_data_zip   = wf_topology,
           docking_output_zip_path     = docking_output_zip_path,
           ambig_restraints_table_path = barnase_barstar_manual_tbl,
           output_haddock_wf_data_zip  = wf_rigidbody,
           properties                  = properties)

0

In [23]:
folder = docking_output_zip_path[:-4]
if os.path.exists(folder):
    shutil.rmtree(folder)
if not os.path.exists(folder):
    os.makedirs(folder)

with zipfile.ZipFile(docking_output_zip_path, 'r') as zip_ref:
    zip_ref.extractall(folder)

In [6]:
import pytraj as pt
import glob

pdb_dir = "data/barnase_barstar/1/docking/"
pdb_files = sorted(glob.glob(f"{pdb_dir}/*.pdb.gz"))
def show_aligned(chain):
    # Get all PDB files and sort them
    # Create a trajectory from the PDB files
    traj = pt.iterload(pdb_files, top=pdb_files[0])
    # Save the trajectory
    # pt.write_traj(f"{pdb_dir}/combined_{chain}_aligned.dcd", traj, overwrite=True)
    pt.align(traj, ref=0, mask=f'::{chain}')
    traj.save(f"{pdb_dir}/combined_{chain}_aligned_clust.pdb", options="model", overwrite=True)
    view = nv.show_pytraj(traj)
    view.layout.width = '100%'
    return view

In [7]:
view1 = show_aligned('B') # barnase
view2 = show_aligned('A') # barstar

# Display the viewer
ipywidgets.HBox([view1, view2])

In [8]:
view1 = nv.show_structure_file(f"{pdb_dir}/combined_A_aligned_clust.pdb", default_representation=False)
view2 = nv.show_structure_file(f"{pdb_dir}/combined_B_aligned_clust.pdb", default_representation=False)
view1.add_ribbon(color='chainIndex')
view2.add_ribbon(color='chainIndex')
view1.layout.width = '100%'
view2.layout.width = '100%'
# Display the viewer
box = ipywidgets.HBox([view1, view2])
display(box)
# Create a dropdown widget
opts = ['All']
opts.extend([pdb_file.split('/')[-1].split('.')[0] for pdb_file in pdb_files])
mdsel = ipywidgets.Dropdown(
    options=opts,
    description='Sel. model:',
    disabled=False,
)
display(mdsel)

def on_dropdown_change(change):
    """Handle dropdown selection changes.
    From https://github.com/nglviewer/nglview/issues/765
    """
    if change['type'] == 'change' and change['name'] == 'value': 
        selected_file = change['new']
        if selected_file=='All':
            view1._remote_call('setSelection', target='compList', args=["*"], 
               kwargs=dict(component_index=0))
            view2._remote_call('setSelection', target='compList', args=["*"], 
               kwargs=dict(component_index=0))
        else:
            # Extract model number from the filename
            model_num = selected_file.split('_')[1]
            print(f"Selected model: {model_num}")
            # Update the view with the selected model
            view1._remote_call('setSelection', target='compList', 
                            args=[f"/{model_num}"], 
                            kwargs=dict(component_index=0))
            # You can also update view2 if needed
            view2._remote_call('setSelection', target='compList', 
                            args=[f"/{model_num}"], 
                            kwargs=dict(component_index=0))

# Register the callback function
mdsel.observe(on_dropdown_change, names='value')

Dropdown(description='Sel. model:', options=('All', 'rigidbody_1', 'rigidbody_10', 'rigidbody_2', 'rigidbody_3…

### 2. CAPRI eval

In [9]:
from biobb_haddock.haddock.capri_eval import capri_eval

output_evaluation_zip_path = f'{out_path}2/caprieval.zip'
wf_caprieval               = f'{out_path}2/wf_caprieval.zip'

capri_eval(input_haddock_wf_data_zip  = wf_rigidbody,
           reference_pdb_path         = complex_pdb_ch,
           output_evaluation_zip_path = output_evaluation_zip_path,
           output_haddock_wf_data_zip = wf_caprieval,
           properties = def_dict())

0

In [26]:
with zipfile.ZipFile(wf_caprieval, 'r') as zip_ref:
    zip_ref.extractall(wf_caprieval[:-4])
    
#webbrowser.open(f"http://0.0.0.0:8000/{wf_caprieval[:-4]}/analysis/2_caprieval_analysis/report.html")
#!python3 -m http.server

In [11]:
tsv_dir = wf_caprieval[:-4]+'/2_caprieval/'
# Load the cluster and single data into pandas DataFrames
cluster_df = pd.read_csv(tsv_dir + 'capri_clt.tsv', sep='\t',comment='#')
single_df = pd.read_csv(tsv_dir + 'capri_ss.tsv', sep='\t',comment='#')

# DockQ: incorrect (<0.23), acceptable (0.23-0.49), medium (0.49-0.80), and high (>=0.80) 
display(single_df.head())
single_df = single_df.sort_values(by='dockq', ascending=False)
display(single_df.head())
display(cluster_df.head())

,model,md5,caprieval_rank,score,irmsd,fnat,lrmsd,ilrmsd,dockq,rmsd,...,dihe,elec,improper,rdcs,rg,sym,total,vdw,vean,xpcs
0,../1_rigidbody/rigidbody_7.pdb,-,1,-37.081,10.519,0.091,17.705,17.544,0.099,10.438,...,0.0,-4.819,0.0,0.0,0.0,0.0,55.489,-31.075,0.0,0.0
1,../1_rigidbody/rigidbody_2.pdb,-,2,-31.161,12.581,0.073,21.207,20.822,0.075,12.862,...,0.0,-6.684,0.0,0.0,0.0,0.0,135.270,-17.045,0.0,0.0
2,../1_rigidbody/rigidbody_3.pdb,-,3,-29.411,12.309,0.073,18.342,18.413,0.088,11.782,...,0.0,-4.963,0.0,0.0,0.0,0.0,120.690,-32.024,0.0,0.0
3,../1_rigidbody/rigidbody_8.pdb,-,4,-28.814,12.281,0.073,18.288,18.420,0.088,11.759,...,0.0,-4.760,0.0,0.0,0.0,0.0,31.295,-34.684,0.0,0.0
4,../1_rigidbody/rigidbody_10.pdb,-,5,-19.919,6.586,0.055,11.109,11.485,0.158,6.243,...,0.0,-15.013,0.0,0.0,0.0,0.0,177.654,-1.754,0.0,0.0


,model,md5,caprieval_rank,score,irmsd,fnat,lrmsd,ilrmsd,dockq,rmsd,...,dihe,elec,improper,rdcs,rg,sym,total,vdw,vean,xpcs
5,../1_rigidbody/rigidbody_1.pdb,-,6,-19.570,3.532,0.145,5.620,6.381,0.331,3.171,...,0.0,-5.610,0.0,0.0,0.0,0.0,33.317,-17.081,0.0,0.0
4,../1_rigidbody/rigidbody_10.pdb,-,5,-19.919,6.586,0.055,11.109,11.485,0.158,6.243,...,0.0,-15.013,0.0,0.0,0.0,0.0,177.654,-1.754,0.0,0.0
7,../1_rigidbody/rigidbody_6.pdb,-,8,-13.935,7.449,0.055,14.592,13.003,0.116,7.690,...,0.0,-7.582,0.0,0.0,0.0,0.0,204.137,-18.965,0.0,0.0
0,../1_rigidbody/rigidbody_7.pdb,-,1,-37.081,10.519,0.091,17.705,17.544,0.099,10.438,...,0.0,-4.819,0.0,0.0,0.0,0.0,55.489,-31.075,0.0,0.0
2,../1_rigidbody/rigidbody_3.pdb,-,3,-29.411,12.309,0.073,18.342,18.413,0.088,11.782,...,0.0,-4.963,0.0,0.0,0.0,0.0,120.690,-32.024,0.0,0.0


,cluster_rank,cluster_id,n,under_eval,score,score_std,irmsd,irmsd_std,fnat,fnat_std,...,bsa_std,desolv,desolv_std,elec,elec_std,total,total_std,vdw,vdw_std,caprieval_rank
0,-,-,10,-,-22.936,7.896,10.268,3.891,0.077,0.045,...,174.841,-8.185,5.828,-5.935,0.691,156.495,111.109,-19.954,7.105,1


In [12]:
import gzip
import shutil
best_pdb = os.path.normpath(os.path.join(tsv_dir, single_df['model'][0]))
# Decompress the .gz file
with gzip.open(best_pdb + '.gz', 'rb') as f_in:
    with open(best_pdb, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

In [13]:
show_pdbs([best_pdb, complex_pdb_ch])

In [14]:
# the reference and the input proteins have diferent number of residues/atoms, 
# so a fit based on rmsd like pytraj does fails

# # TODO: meter en structure utils
from Bio.PDB import PDBParser, PDBIO
from Bio.PDB.cealign import CEAligner

# Parse the structures
parser = PDBParser(QUIET=True)
structure1 = parser.get_structure("complex_pdb_ch", complex_pdb_ch)
structure2 = parser.get_structure("best_pdb", best_pdb)
    
# Perform CE alignment
aligner = CEAligner()
aligner.set_reference(structure1)
aligner.align(structure2)

# Save structure2 to a PDB file
output_pdb_path = f"{out_path}aligned_structure2.pdb"
io = PDBIO()
io.set_structure(structure2)
io.save(output_pdb_path)

In [15]:
view = nv.show_structure_file(output_pdb_path)
view.add_component(complex_pdb_ch)
view.clear()
view.component_0.add_cartoon(selection=f':{barnase_ch}', color='red')
view.component_0.add_cartoon(selection=f':{barstar_ch}', color='pink')
view.component_1.clear()
view.component_1.add_cartoon(selection=f':{complex_ch[0]}', color='blue')
view.component_1.add_cartoon(selection=f':{complex_ch[-1]}', color='cyan')
view

NGLWidget()

### 3. Extend docking

In [19]:
# Files are relative to the input_haddock_wf_data_zip
cfg ="""
[seletop]
select = 25

[caprieval]
reference_fname = "./data/2_caprieval/1BRS_ch.pdb"

[flexref]
tolerance = 5
ambig_fname = "./data/1_rigidbody/barnase_barstar_manual.tbl"

[caprieval]
reference_fname = "./data/2_caprieval/1BRS_ch.pdb"

[emref]
tolerance = 5
ambig_fname = "./data/1_rigidbody/barnase_barstar_manual.tbl"

[caprieval]
reference_fname = "./data/2_caprieval/1BRS_ch.pdb"
# ====================================================================
"""
haddock_config_path        = f'{out_path}docking-barnase-barstar.cfg'

with open(haddock_config_path, 'w') as config_file:
    config_file.write(cfg)

In [20]:
from biobb_haddock.haddock.haddock3_extend import haddock3_extend

output_haddock_wf_data_zip = f'{out_path}3/extend_wf.zip'  

haddock3_extend(input_haddock_wf_data_zip  = wf_caprieval,
                haddock_config_path        = haddock_config_path,
                output_haddock_wf_data_zip = output_haddock_wf_data_zip,
                properties = def_dict())

0

In [25]:
with zipfile.ZipFile(output_haddock_wf_data_zip, 'r') as zip_ref:
    zip_ref.extractall(output_haddock_wf_data_zip[:-4])
    
webbrowser.open("http://localhost:8888/edit/biobb_wf_haddock/notebooks/data/barnase_barstar/3/extend_wf/analysis/8_caprieval_analysis/report.html")
#!python3 -m http.server

True

In [ ]:
from IPython.display import IFrame
#display(IFrame(src='/path/to/your/file.html', width=800, height=600))

In [22]:
tsv_dir = output_haddock_wf_data_zip[:-4]+'/8_caprieval/'
# Load the cluster and single data into pandas DataFrames
cluster_df = pd.read_csv(tsv_dir + 'capri_clt.tsv', sep='\t',comment='#')
single_df = pd.read_csv(tsv_dir + 'capri_ss.tsv', sep='\t',comment='#')

# DockQ: incorrect (<0.23), acceptable (0.23-0.49), medium (0.49-0.80), and high (>=0.80) 
display(single_df.head())
single_df = single_df.sort_values(by='dockq', ascending=False)
display(single_df.head())
display(cluster_df.head())

,model,md5,caprieval_rank,score,irmsd,fnat,lrmsd,ilrmsd,dockq,rmsd,...,dihe,elec,improper,rdcs,rg,sym,total,vdw,vean,xpcs
0,../7_emref/emref_7.pdb,-,1,-117.701,12.814,0.073,18.964,18.776,0.085,11.950,...,0.0,-286.254,0.0,0.0,0.0,0.0,-332.036,-47.892,0.0,0.0
1,../7_emref/emref_1.pdb,-,2,-103.956,10.943,0.073,18.897,18.034,0.086,10.878,...,0.0,-226.571,0.0,0.0,0.0,0.0,-250.697,-38.348,0.0,0.0
2,../7_emref/emref_2.pdb,-,3,-87.783,12.370,0.055,20.782,19.740,0.071,12.726,...,0.0,-222.792,0.0,0.0,0.0,0.0,-250.085,-36.871,0.0,0.0
3,../7_emref/emref_5.pdb,-,4,-83.329,6.200,0.073,10.348,11.039,0.177,5.858,...,0.0,-360.748,0.0,0.0,0.0,0.0,-376.941,-18.320,0.0,0.0
4,../7_emref/emref_3.pdb,-,5,-81.900,12.479,0.091,18.850,18.578,0.091,11.998,...,0.0,-252.467,0.0,0.0,0.0,0.0,-276.038,-29.830,0.0,0.0


,model,md5,caprieval_rank,score,irmsd,fnat,lrmsd,ilrmsd,dockq,rmsd,...,dihe,elec,improper,rdcs,rg,sym,total,vdw,vean,xpcs
6,../7_emref/emref_6.pdb,-,7,-78.496,3.782,0.145,6.579,7.105,0.302,3.413,...,0.0,-302.977,0.0,0.0,0.0,0.0,-298.292,-20.686,0.0,0.0
3,../7_emref/emref_5.pdb,-,4,-83.329,6.200,0.073,10.348,11.039,0.177,5.858,...,0.0,-360.748,0.0,0.0,0.0,0.0,-376.941,-18.320,0.0,0.0
5,../7_emref/emref_8.pdb,-,6,-81.642,7.256,0.127,14.158,12.438,0.144,7.550,...,0.0,-319.729,0.0,0.0,0.0,0.0,-317.847,-21.555,0.0,0.0
4,../7_emref/emref_3.pdb,-,5,-81.900,12.479,0.091,18.850,18.578,0.091,11.998,...,0.0,-252.467,0.0,0.0,0.0,0.0,-276.038,-29.830,0.0,0.0
1,../7_emref/emref_1.pdb,-,2,-103.956,10.943,0.073,18.897,18.034,0.086,10.878,...,0.0,-226.571,0.0,0.0,0.0,0.0,-250.697,-38.348,0.0,0.0


,cluster_rank,cluster_id,n,under_eval,score,score_std,irmsd,irmsd_std,fnat,fnat_std,...,bsa_std,desolv,desolv_std,elec,elec_std,total,total_std,vdw,vdw_std,caprieval_rank
0,-,-,10,-,-80.373,20.439,12.036,0.633,0.068,0.015,...,178.9,-6.904,9.385,-227.175,16.368,-241.623,31.77,-29.544,10.011,1
